# 0. 개관

- 사용 더미 데이터: 리뷰 3건(각 리뷰는 2~3문장), 국가/플랫폼/리뷰 텍스트 포함

- 파이프라인 단계:

  - 언어 감지 및 간단 영문화(더미 매핑) — langdetect 사용, NLLB 모델로(`facebook/nllb-200-distilled-600M`)로 감지된 언어 → 영어(`eng_Latn`) 번역

  - 리뷰 문장 분해(문장 단위 분리)

  - `nlptown/bert-base-multilingual-uncased-sentiment` 모델로 각 문장 감정 점수(1~5) 산출: 점수 기준으로 `negative`, `neutral`, `positive` 라벨 부여

  - 리뷰 단위 감정 판단: 문장 점수 평균과 부정 문장 비율 계산

  - 리뷰 전체 코퍼스에 대해 KeyBERT로 대표 키워드 추출

  - TF-IDF 벡터화 + KMeans로 클러스터링

  - 국가/플랫폼별 집계(긍부정 분포, irritation 키워드 빈도) 및 간단한 이슈 탐지 룰(부정 비율 30% 초과, 또는 자극 키워드 탐지 시 알림)

## 파일 열기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
df_japan = pd.read_excel('oliveyoung_일본_20251221_141513.xlsx')
df_usa = pd.read_excel('oliveyoung_미국_20251221_045252.xlsx')
df_china = pd.read_excel('oliveyoung_중국_20251221_174445.xlsx')

In [ ]:
df_japan.head(3)

,country,major_category,sub_category,rank,product_name,brand,discount_rate,price,rating,ingredients,description,review_content,review_date,review_rating,review_detail_ratings,product_url
0,Japan,Skincare,Moisturizers,1,★2025 アワード★ Anua PDRNヒアルロン酸カプセル100セラム30ml詰め替え限...,アヌア,53%,"¥8,176",4.8,PDRNが配合されて肌調子を健康に整え、艶を与えます。\n11種ヒアルロン酸を配合し、肌の奥...,一滴も希釈せず、肌の奥から湧き上がるツヤプランピングセラムです。うるおいプランピング効果を与...,これは私の2回目の購入です！本当に素晴らしいです！べたつかず、保湿効果も抜群で、吸収もとても...,2025/12/20,5,"{""\u7d20\u65e9\u304f\u6d78\u900f"": 5.0, ""\u30e...",https://global.oliveyoung.com/product/detail?p...
1,Japan,Skincare,Moisturizers,1,★2025 アワード★ Anua PDRNヒアルロン酸カプセル100セラム30ml詰め替え限...,アヌア,53%,"¥8,176",4.8,PDRNが配合されて肌調子を健康に整え、艶を与えます。\n11種ヒアルロン酸を配合し、肌の奥...,一滴も希釈せず、肌の奥から湧き上がるツヤプランピングセラムです。うるおいプランピング効果を与...,使用後は、べたつき感がなく、とても快適でしっとりします。塗布後、メイクが非常によく密着します...,2025/12/20,5,"{""\u7d20\u65e9\u304f\u6d78\u900f"": 5.0, ""\u30e...",https://global.oliveyoung.com/product/detail?p...
2,Japan,Skincare,Moisturizers,1,★2025 アワード★ Anua PDRNヒアルロン酸カプセル100セラム30ml詰め替え限...,アヌア,53%,"¥8,176",4.8,PDRNが配合されて肌調子を健康に整え、艶を与えます。\n11種ヒアルロン酸を配合し、肌の奥...,一滴も希釈せず、肌の奥から湧き上がるツヤプランピングセラムです。うるおいプランピング効果を与...,韓国でこの美容液を購入して自分の肌質にとても合っているので今回はオリーブヤングで直接購入しま...,2025/12/20,5,"{""\u7d20\u65e9\u304f\u6d78\u900f"": 5.0, ""\u30e...",https://global.oliveyoung.com/product/detail?p...


## Download

In [ ]:
!pip install langdetect transformers keybert sentence-transformers scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.6 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=9cbb539cd31e2b4a6e070bb8a42cc5439d36d82a851873398aeb1d7a87bfaf5b
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [ ]:
from pprint import pprint
from collections import defaultdict, Counter
import re
from transformers import pipeline
from langdetect import detect
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [ ]:
translator = pipeline(
    "translation",
    model="facebook/nllb-200-distilled-600M"
)

sentiment_pipe = pipeline(
    "sentiment-analysis",
    model="nlptown/bert-base-multilingual-uncased-sentiment"
)

# KeyBERT 키워드 추출용 임베딩 모델
embedding_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")
kw_model = KeyBERT(embedding_model)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# 1. TEST

 ## 언어 번역 (일 -> 영)

In [ ]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'en'

lang_map = {
    "ko": "kor_Hang",
    "ja": "jpn_Jpan",
    "en": "eng_Latn",
    "fr": "fra_Latn",
    "de": "deu_Latn",
    "id": "ind_Latn",
}

# TODO: Google/Azure 같은 외부 번역을 붙일지
def normalize_language_and_translate(text):
    lang_code = detect_language(text)
    src_lang = lang_map.get(lang_code, "eng_Latn")
    translated = translator(text, src_lang=src_lang, tgt_lang="eng_Latn")[0]["translation_text"]
    return {"lang": src_lang, "text_en": translated}

In [ ]:
# 첫 번째 리뷰로 테스트
sample_review = df_japan['review_content'].iloc[0]
print(f"원문: {sample_review}")

result = normalize_language_and_translate(sample_review)
print(f"언어: {result['lang']}")
print(f"번역: {result['text_en']}")

원문: これは私の2回目の購入です！本当に素晴らしいです！べたつかず、保湿効果も抜群で、吸収もとても早いです。このセットには、リフィル、クリーム、さらにコットンパッドも付いており、価格もとてもリーズナブルです！
언어: jpn_Jpan
번역: This is my second purchase! It's really great! It's all good, it's very humid, it's very absorbent, and it's very fast. This set comes with a filler, a cream, and a cotton pad, and it's very affordable!


In [ ]:
print(f"원문: {df_japan['product_name'].iloc[0]}")
result = normalize_language_and_translate(df_japan['product_name'].iloc[0])
print(f"번역: {result['text_en']}")

print(f"원문: {df_japan['brand'].iloc[0]}")
result = normalize_language_and_translate(df_japan['brand'].iloc[0])
print(f"번역: {result['text_en']}")

print(f"원문: {df_japan['ingredients'].iloc[0]}")
result = normalize_language_and_translate(df_japan['ingredients'].iloc[0])
print(f"번역: {result['text_en']}")

print(f"원문: {df_japan['description'].iloc[0]}")
result = normalize_language_and_translate(df_japan['description'].iloc[0])
print(f"번역: {result['text_en']}")

## Pipeline (리뷰 -> 키워드 추출 부분)

In [ ]:
# =============================================================================
# 올리브영 리뷰 분석 파이프라인 - 리뷰 키워드 추출 (테스트용 10개)
# =============================================================================

import pandas as pd
import numpy as np
from transformers import pipeline
from langdetect import detect
from keybert import KeyBERT
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import re
from collections import defaultdict

# -----------------------------------------------------------------------------
# 1. 모델 초기화
# -----------------------------------------------------------------------------

translator = pipeline("translation", model="facebook/nllb-200-distilled-600M")
kw_model = KeyBERT()

lang_map = {
    'ja': 'jpn_Jpan',
    'ko': 'kor_Hang',
    'en': 'eng_Latn',
    'zh-cn': 'zho_Hans',
}

# -----------------------------------------------------------------------------
# 2. 번역 함수
# -----------------------------------------------------------------------------

def normalize_language_and_translate(text):
    try:
        if pd.isna(text) or str(text).strip() == '':
            return {'lang': 'unknown', 'text_en': ''}

        lang_code = detect(str(text))

        if lang_code == 'en':
            return {'lang': 'eng_Latn', 'text_en': str(text)}

        src_lang = lang_map.get(lang_code, 'jpn_Jpan')
        translated = translator(
            str(text),
            src_lang=src_lang,
            tgt_lang='eng_Latn',
            max_length=512
        )[0]['translation_text']

        return {'lang': src_lang, 'text_en': translated}
    except:
        return {'lang': 'unknown', 'text_en': str(text)}

# -----------------------------------------------------------------------------
# 3. 문장 분리 함수
# -----------------------------------------------------------------------------

def split_sentences(text):
    if not text:
        return []
    sentences = re.split(r'[.!?]+', text)
    return [s.strip() for s in sentences if s.strip()]

# -----------------------------------------------------------------------------
# 4. 문장별 감정 점수 (더미 함수 - 실제로는 감정분석 모델 필요)
# -----------------------------------------------------------------------------

def sentiment_of_sentence(sentence):
    # 실제로는 감정분석 모델 사용
    # 여기서는 임시로 3점 반환
    return 3

# -----------------------------------------------------------------------------
# 5. 리뷰 전체 감정 판단 (더미 함수)
# -----------------------------------------------------------------------------

def sentiment_of_review(sentences):
    # 실제로는 감정분석 모델 사용
    # 여기서는 임시로 neutral 반환
    return 'neutral'

# -----------------------------------------------------------------------------
# 6. KeyBERT 키워드 추출
# -----------------------------------------------------------------------------

def extract_keywords_keybert(text, top_n=5):
    if not text or len(text.strip()) < 10:
        return []

    try:
        keywords = kw_model.extract_keywords(
            text,
            keyphrase_ngram_range=(1, 2),
            stop_words='english',
            top_n=top_n
        )
        return [kw[0] for kw in keywords]
    except:
        return []

# -----------------------------------------------------------------------------
# 7. 제품별 리뷰 파이프라인
# -----------------------------------------------------------------------------

def run_review_pipeline(df, top_n_reviews=10):
    """
    제품별 리뷰 분석 파이프라인

    Args:
        df: 입력 데이터프레임
        top_n_reviews: 처리할 리뷰 개수

    Returns:
        dict: 처리된 리뷰 결과
    """

    # 상위 N개 리뷰만 선택
    df_sample = df.head(top_n_reviews).copy()

    processed = []

    print("===== PROCESSING REVIEWS =====")

    for idx, row in df_sample.iterrows():
        review_id = f"r{idx+1}"
        country = row.get('country', 'Unknown')
        platform = 'oliveyoung_global'
        review_text = row.get('review_content', '')

        # 번역
        norm = normalize_language_and_translate(review_text)

        # 문장 분리
        sentences = split_sentences(norm['text_en'])

        # 문장별 감정 점수
        sentence_records = []
        for s in sentences:
            score = sentiment_of_sentence(s)
            sentence_records.append({'sentence': s, 'score': score})

        # 리뷰 전체 감정
        review_sentiment = sentiment_of_review(sentences)

        # 키워드 추출
        keywords = extract_keywords_keybert(norm['text_en'], top_n=5)

        processed.append({
            'id': review_id,
            'country': country,
            'platform': platform,
            'lang': norm['lang'],
            'text_en': norm['text_en'],
            'sentences': sentence_records,
            'review_sentiment': review_sentiment,
            'kw_keybert': keywords,
            'cluster': 0  # 클러스터는 나중에 계산
        })

    # 클러스터링 (3개 이상일 때만)
    if len(processed) >= 3:
        corpus = [p['text_en'] for p in processed]
        vectorizer = TfidfVectorizer(max_df=0.9, min_df=1, ngram_range=(1, 2), stop_words='english')
        X = vectorizer.fit_transform(corpus)
        n_clusters = max(1, min(3, len(corpus)))
        km = KMeans(n_clusters=n_clusters, random_state=42)
        km.fit(X)
        labels = km.labels_

        for i, p in enumerate(processed):
            p['cluster'] = int(labels[i])

    return processed

# -----------------------------------------------------------------------------
# 8. 결과 출력 함수
# -----------------------------------------------------------------------------

def print_processed_reviews(processed):
    """
    처리된 리뷰 결과를 예시 형식으로 출력
    """
    print("\n===== PROCESSED REVIEWS =====")

    for p in processed:
        print(f"\nID: {p['id']} | country: {p['country']} | platform: {p['platform']} | lang: {p['lang']} | cluster: {p['cluster']}")

        for s in p['sentences']:
            print(f"  - {s}")

        print(f"  Review sentiment: {p['review_sentiment']}")
        print(f"  KeyBERT keywords: {p['kw_keybert']}")

# -----------------------------------------------------------------------------
# 9. 결과를 테이블로 저장
# -----------------------------------------------------------------------------

def save_results_to_table(processed, output_file='review_analysis_results.xlsx'):
    """
    처리된 리뷰 결과를 테이블 형태로 저장

    Args:
        processed: 처리된 리뷰 리스트
        output_file: 출력 파일명
    """
    rows = []

    for p in processed:
        # 문장들을 하나의 문자열로 합치기
        sentences_text = ' | '.join([f"{s['sentence']} (score: {s['score']})" for s in p['sentences']])

        row = {
            'review_id': p['id'],
            'country': p['country'],
            'platform': p['platform'],
            'language': p['lang'],
            'translated_text': p['text_en'],
            'sentences': sentences_text,
            'review_sentiment': p['review_sentiment'],
            'keywords': ', '.join(p['kw_keybert']),
            'cluster': p['cluster']
        }
        rows.append(row)

    result_df = pd.DataFrame(rows)
    result_df.to_excel(output_file, index=False)
    print(f"\n결과가 {output_file}에 저장되었습니다")

    return result_df

# =============================================================================
# 실행
# =============================================================================

if __name__ == "__main__":
    # 데이터 로드
    df = pd.read_excel('oliveyoung_미국_20251221_045252.xlsx')

    # 파이프라인 실행 (10개만)
    processed = run_review_pipeline(df, top_n_reviews=10)

    # 결과 출력
    print_processed_reviews(processed)

    # 결과 저장
    result_df = save_results_to_table(processed, 'review_analysis_results.xlsx')

    print("\n테스트 완료")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


===== PROCESSING REVIEWS =====

===== PROCESSED REVIEWS =====

ID: r1 | country: USA | platform: oliveyoung_global | lang: eng_Latn | cluster: 2
  - {'sentence': 'This is my second purchase', 'score': 3}
  - {'sentence': "It's truly amazing", 'score': 3}
  - {'sentence': "It's not sticky at all, the moisturizing effect is superb, and it absorbs very quickly", 'score': 3}
  - {'sentence': 'This set also includes a refill, cream, and even a cotton pad, and the price is very reasonable', 'score': 3}
  Review sentiment: neutral
  KeyBERT keywords: ['sticky moisturizing', 'sticky', 'amazing sticky', 'moisturizing effect', 'moisturizing']

ID: r2 | country: USA | platform: oliveyoung_global | lang: eng_Latn | cluster: 0
  - {'sentence': 'It feels very comfortable and moisturizing after use, without any sticky feeling', 'score': 3}
  - {'sentence': 'Makeup adheres very well after application', 'score': 3}
  - {'sentence': 'After using it at night, my skin felt very moisturized the next mornin

In [ ]:
def run_pipeline(reviews):
    processed = []
    for r in reviews:
        norm = normalize_language_and_translate(r['review'])
        sentences = split_sentences(norm['text_en'])

        sentence_records = []
        for s in sentences:
            score = sentiment_of_sentence(s)  # 1~5 정수 반환
            # 문장별 감정 라벨은 따로 만들지 않고 점수만 저장
            sentence_records.append({'sentence': s, 'score': score})

        review_sentiment = sentiment_of_review(sentences)  # 전체 리뷰 감정 판단

        processed.append({
            'id': r['id'],
            'country': r['country'],
            'platform': r['platform'],
            'lang': norm['lang'],
            'text_en': norm['text_en'],
            'sentences': sentence_records,
            'review_sentiment': review_sentiment,
        })

    corpus = [p['text_en'] for p in processed]
    kw_results = keybert_keywords(corpus, top_n=5)
    for i, p in enumerate(processed):
        p['kw_keybert'] = kw_results.get(i, [])

    vectorizer = TfidfVectorizer(max_df=0.9, min_df=1, ngram_range=(1, 2), stop_words='english')
    X = vectorizer.fit_transform(corpus)
    n_clusters = max(1, min(3, len(corpus)))
    km = KMeans(n_clusters=n_clusters, random_state=42)
    km.fit(X)
    labels = km.labels_
    for i, p in enumerate(processed):
        p['cluster'] = int(labels[i])

    agg = defaultdict(lambda: {'positive':0, 'neutral':0, 'negative':0, 'total_sentences':0})
    for p in processed:
        for s in p['sentences']:
            # 문장 점수 기반 감정 라벨링 집계
            score = s['score']
            if score <= 2:
                sentiment = 'negative'
            elif score == 3:
                sentiment = 'neutral'
            else:
                sentiment = 'positive'

            agg_key = (p['country'], p['platform'])
            agg[agg_key][sentiment] += 1
            agg[agg_key]['total_sentences'] += 1

    alerts = []
    for k, v in agg.items():
        neg_rate = v['negative'] / v['total_sentences'] if v['total_sentences'] else 0
        if neg_rate > 0.3:
            alerts.append({'scope': k, 'type': 'negative_spike', 'neg_rate': neg_rate})

    return {'processed': processed, 'aggregations': dict(agg), 'alerts': alerts}


# 필요한 것

- 현재 KeyBert로 키워드 추출만 한 거라 도움은 되지만 세분화가 필요함. 뷰티 도메인에 맞게 KeyBert로 중요 키워드를 자동으로 뽑고, NER+화장품사전리스트로 성분/제형으로 매칭을 하든지 NER을 뷰티 도메인으로 학습시킬 것

- 키워드 위주로 클러스터링도 해야 함 현재로는 리뷰끼리 비슷한지를 분류하는 느낌

- 번역을 더 자세한 걸로 할지

- 현재 방향성은 이 제품에 대한 이 리뷰가 긍정, 부정, 무난인지(문장이 여러 개면 전체적인 평점인 것) 판단하는 것이므로 이걸로 제품이 긍정적인지~를 알 수 있을 듯

- 시간, 지역, 플랫폼 별 필터링/분석 좀 세분화 필요

- db 아직 연결 안 됨 (구축 후 가져오도록 해야 할 듯?)

- 현재는 제대로 된 리뷰 문장이 있을 경우(이건 올리브영 댓글이 필요할 것 같음. 인스타로는 댓글을 못 보고 캡션에도 필요한 정보가 없긴 함)

- 룰 기반 이나 NER 모델 사용하는데 이걸 키워드 다 적을 수가 있을지